In [2]:
import tkinter as tk
from tkinter import messagebox

# Onsight Competition Score Application

## Goals
What should this application be able to do?  

**Basics**  
* Be able to enter the following per boulder:
    * Top (if yes how many attempts)
    * Zone (if yes how many attempts)
    * Attempts (total if neither zone nor top achieved)
* Be able to enter Tops/Zones/Attempts for each boulder in the round.
* Create leaderboard ranking based on entered information.

**Advanced**  
* Somehow link functionality to RO_app. Where the program takes information like the number of boulders per round, and list of competitors. (May need to overhaul the layout and present a more OOP style to the backend.)
* Active controls once base program is set up.

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox

# OOP Scoring System logic
class Scores:
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
    
    class Zone:
        def __init__(self, zones = 0, attempts = 0):
            self.zones = zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.zones} Z    {self.attempts} A"

    class LowZone:
        def __init__(self, low_zones = 0, attempts = 0):
            self.low_zones = low_zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.low_zones} LZ    {self.attempts} A"

    def __init__(self):
        self.tops = Scores.Top(0, 0)
        self.zones = Scores.Zone(0, 0)
        self.low_zones = Scores.LowZone(0, 0)

class Climber:
    def __init__(self, name, category):
        self.name = name
        self.scores = Scores()
        self.category = category

    def __str__(self):
        return f"{self.name}:\n{self.scores.tops}\n{self.scores.zones}\n{self.scores.low_zones}"
    
    def delete(self):
        """Deletes the climber from the leaderboard"""
        del self
    
class Leaderboard():
    def __init__(self):
        self.climbers = []

    def add_climber(self, *climbers):
        """Adds a climber to the leaderboard"""
        self.climbers.extend(climbers)
    
    def rank_climbers(self):
        """Sorts climbers by their scores. Key currently matches the USA Climbing Isolation scoring system."""
        def sort_key(climber):
            return (-climber.scores.tops.tops, -climber.scores.zones.zones, -climber.scores.low_zones.low_zones, 
            climber.scores.tops.attempts, climber.scores.zones.attempts, climber.scores.low_zones.attempts)
        self.climbers.sort(key=sort_key)
    
def create_climber(name, category, tops, zones, low_zones = (0, 0)):
    """Creates a climber object with the given name, category, and scores. Low zones are optional and scores are taken as tuples."""
    climber = Climber(name, category)
    climber.scores.tops = Scores.Top(*tops)
    climber.scores.zones = Scores.Zone(*zones)
    climber.scores.low_zones = Scores.LowZone(*low_zones)
    return climber

# tkinter window
class ScoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        self.leaderboard = Leaderboard()
        
        # Main Frame
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.main_frame.rowconfigure(0, weight= 1)
        self.main_frame.columnconfigure(0, weight= 1)
        self.main_frame.columnconfigure(1, weight= 1)

        # Resizing to fit the frame
        self.root.columnconfigure(0, weight= 1)
        self.root.rowconfigure(0, weight= 1)

        # Creating Left and Right sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons"""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, sticky= 'nw')

        # Left Frame Resizing
        self.left_frame.columnconfigure(0, weight= 1)
        self.left_frame.columnconfigure(1, weight= 1)
        self.left_frame.rowconfigure(0, weight= 1)
        self.left_frame.rowconfigure(5, weight= 1)

        # Labels and Entry Fields
        labels = ["Climber Name", "Category", "Tops (count, attempts)", "Zones (count, attempts)", "Low Zones (count, attempts)"]
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text= label + ":").grid(row=i, column=0, padx=5, pady=5)
            entry = ttk.Entry(self.left_frame)
            entry.grid(row= i, column= 1, padx=5, pady=5, sticky= "w")
            self.entries[label] = entry #stores in dictionary for easy access
        
        # Leftside Buttons
        ttk.Button(self.left_frame, text="Add Climber", command=self.add_climber).grid(row= 5, column= 0, columnspan= 2, pady= 10)

        # Edit/Remove Climber Buttons
        add_remove_buttons = [
            ("Edit Climber", self.edit_climber),
            ("Remove Climber", self.remove_climber)
        ]
        for i, (text, command) in enumerate(add_remove_buttons):
            ttk.Button(self.left_frame, text= text, command= command).grid(row= 6, column= i, columnspan= 2, padx= 10, pady= 10, sticky= "w")

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky= 'nsew')

        #letting right frame expand to fill the space
        self.main_frame.columnconfigure(1, weight= 1)
        self.right_frame.rowconfigure(0, weight= 1)
        self.right_frame.columnconfigure(0, weight= 1)

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=30)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady=10, sticky= "nsew")

        # Rightside Buttons
        self.leaderboard_buttons = ttk.Frame(self.right_frame)
        self.leaderboard_buttons.grid(row=1, column=0, columnspan=2, pady=10)
        ttk.Button(self.leaderboard_buttons, text="Clear Leaderboard", command=self.clear_leaderboard_ask).pack(side= "left", padx= 10, pady= 10)
        ttk.Button(self.leaderboard_buttons, text="Show Leaderboard", command=self.show_leaderboard).pack(side= "left", padx= 10, pady= 10)

    # Functions and Logic
    def clear_entries(self):
        """Clears all entry fields"""
        for entry in self.entries.values():
            entry.delete(0, tk.END)

    def remove_climber(self):
        """Removes a climber from the Leaderboard"""
        try:
            name = self.entries["Climber Name"].get()
            found = False
            if name == "":
                raise ValueError("Climber Name cannot be empty")
            for climber in self.leaderboard.climbers[:]:
                if climber.name == name:
                    self.leaderboard.climbers.remove(climber)
                    climber.delete()
                    found = True
                    break
            if found:
                messagebox.showinfo("Success", f"{name} has been removed.")
                self.clear_entries()
                self.show_leaderboard()
            else:
                raise ValueError(f"Climber {name} not found in the leaderboard.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def add_climber(self):
        """Adds a climber to the current leaderboard. Raises errors if the input is invalid"""
        try:
            name = self.entries["Climber Name"].get()
            if name == "":
                raise ValueError("Name cannot be empty")
            if name in [climber.name for climber in self.leaderboard.climbers]:
                raise ValueError(f"{name} already exists in the leaderboard")
            category = self.entries["Category"].get()
            tops = tuple(map(int, self.entries["Tops (count, attempts)"].get().split(',')))
            zones = tuple(map(int, self.entries["Zones (count, attempts)"].get().split(',')))
            low_zones = tuple(map(int, self.entries["Low Zones (count, attempts)"].get().split(',')) if self.entries["Low Zones (count, attempts)"].get() else (0, 0))
   
            climber = create_climber(name, category, tops, zones, low_zones)
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"{name} added to the leaderboard!")
            self.clear_entries()
            self.show_leaderboard()
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {e}")

    def show_leaderboard(self):
        """Displays the current leaderboard, with rankings"""
        self.leaderboard.rank_climbers()
        self.leaderboard_text.delete(1.0, tk.END)
        for climber in self.leaderboard.climbers:
            self.leaderboard_text.insert(tk.END, f"{climber}\n\n")

    def edit_climber(self):
        """Edits the score of an existing climber"""
        try: 
            name = self.entries["Climber Name"].get()          
            category = self.entries["Category"].get()
            tops = tuple(map(int, self.entries["Tops (count, attempts)"].get().split(',')))
            zones = tuple(map(int, self.entries["Zones (count, attempts)"].get().split(',')))
            low_zones = tuple(map(int, self.entries["Low Zones (count, attempts)"].get().split(','))) if self.entries["Low Zones (count, attempts)"].get() else (0, 0)

            for climber in self.leaderboard.climbers:
                if climber.name == name:
                    climber.scores.tops = Scores.Top(*tops)
                    climber.scores.zones = Scores.Zone(*zones)
                    climber.scores.low_zones = Scores.LowZone(*low_zones)
                    messagebox.showinfo("Success", f"{name} has been updated.")
                    self.clear_entries()
                    self.show_leaderboard()
                    return
        except Exception as e:
            messagebox.showerror("Error", f"Climber {name} not found. {e}")

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries"""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.clear_leaderboard()
        
    def clear_leaderboard(self):
        """Clears the leaderboard values"""
        self.leaderboard.climbers = []
        self.leaderboard_text.delete(1.0, tk.END)
        messagebox.showinfo("Success", "Leaderboard has been cleared.")

if __name__ == "__main__":
    root = tk.Tk()
    app = ScoringApp(root)
    root.mainloop()

In [15]:
# tkinter window
import tkinter as tk
from tkinter import ttk, messagebox

class ScoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        self.leaderboard = Leaderboard()

        # Main Frame
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.grid(column=0, row=0, padx=20, pady=20, sticky="nsew")

        # Make sure the window resizes correctly
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)
        self.main_frame.columnconfigure(1, weight= 1)
        self.main_frame.rowconfigure(0, weight= 1)

        # Creating Left and Right sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons"""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, sticky="nsew")

        # Sizing information
        self.main_frame.columnconfigure(0, weight= 1)
        self.left_frame.rowconfigure(0, weight= 1)
        self.left_frame.rowconfigure(7, weight= 1)

        # Labels and Entry Fields
        labels = ["Climber Name:", "Category:", "Tops (count, attempts):",
                  "Zones (count, attempts):", "Low Zones (count, attempts):"]

        self.entries = {}

        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text=label).grid(row=i, column=0, padx=5, pady=5, sticky="e")
            entry = ttk.Entry(self.left_frame)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky="w")
            self.entries[label] = entry  # Store in dictionary

        # Left-side Buttons (Properly placed inside self.left_frame)
        ttk.Button(self.left_frame, text="Add Climber", command=self.add_climber).grid(row=5, column=0, columnspan=2, pady=10)
        ttk.Button(self.left_frame, text="Edit Climber", command=self.edit_climber).grid(row=6, column=0, padx=5, pady=10, sticky="w")
        ttk.Button(self.left_frame, text="Remove Climber").grid(row=6, column=1, padx=5, pady=10, sticky="e")

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky="nsew")

        # Configure right frame to expand
        self.main_frame.columnconfigure(1, weight=1)
        self.right_frame.rowconfigure(0, weight=1)
        self.right_frame.columnconfigure(0, weight=1)

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=50)
        self.leaderboard_text.grid(row=0, column=0, columnspan=2, pady=10, sticky="nsew")

        # Rightside Buttons (Placed correctly)
        self.leaderboard_buttons = ttk.Frame(self.right_frame)
        self.leaderboard_buttons.grid(row=1, column=0, columnspan=2, pady=10)

        ttk.Button(self.leaderboard_buttons, text="Show Leaderboard", command=self.show_leaderboard).pack(side="left", padx=5)
        ttk.Button(self.leaderboard_buttons, text="Clear Leaderboard", command=self.clear_leaderboard_ask).pack(side="left", padx=5)

    # Functions and Logic
    def clear_entries(self):
        """Clears all but category entry fields"""
        for entry in self.entries.values():
            entry.delete(0, tk.END)

    def add_climber(self):
        """Adds a climber to the current leaderboard"""
        try:
            name = self.entries["Climber Name:"].get()
            category = self.entries["Category:"].get()
            tops = tuple(map(int, self.entries["Tops (count, attempts):"].get().split(',')))
            zones = tuple(map(int, self.entries["Zones (count, attempts):"].get().split(',')))
            low_zones = tuple(map(int, self.entries["Low Zones (count, attempts):"].get().split(','))) if self.entries["Low Zones (count, attempts):"].get() else (0, 0)
            
            climber = create_climber(name, category, tops, zones, low_zones)
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"{name} added to the leaderboard!")
            self.clear_entries()
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {e}")

    def show_leaderboard(self):
        """Displays the current leaderboard, with rankings"""
        self.leaderboard.rank_climbers()
        self.leaderboard_text.delete(1.0, tk.END)
        for climber in self.leaderboard.climbers:
            self.leaderboard_text.insert(tk.END, f"{climber}\n\n")

    def edit_climber(self):
        """Edits the score of an existing climber"""
        try: 
            name = self.entries["Climber Name:"].get()
            category = self.entries["Category:"].get()
            tops = tuple(map(int, self.entries["Tops (count, attempts):"].get().split(',')))
            zones = tuple(map(int, self.entries["Zones (count, attempts):"].get().split(',')))
            low_zones = tuple(map(int, self.entries["Low Zones (count, attempts):"].get().split(','))) if self.entries["Low Zones (count, attempts):"].get() else (0, 0)

            for climber in self.leaderboard.climbers:
                if climber.name == name:
                    climber.scores.tops = Scores.Top(*tops)
                    climber.scores.zones = Scores.Zone(*zones)
                    climber.scores.low_zones = Scores.LowZone(*low_zones)
                    messagebox.showinfo("Success", f"{name} has been updated.")
                    self.clear_entries()
                    return
        except Exception as e:
            messagebox.showerror("Error", f"invalid input {e}")

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries"""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.clear_leaderboard()
        
    def clear_leaderboard(self):
        """Clears the leaderboard values"""
        self.leaderboard.climbers = []
        self.leaderboard_text.delete(1.0, tk.END)
        messagebox.showinfo("Success", "Leaderboard has been cleared.")

if __name__ == "__main__":
    root = tk.Tk()
    app = ScoringApp(root)
    root.mainloop()


In [ ]:
# old layout that still works but isn't OOP
# tkinter window
import tkinter as tk
from tkinter import ttk, messagebox

class ScoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        self.leaderboard = Leaderboard()
        
        # Main Frame
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")

        # Resizing to fit the frame
        self.root.columnconfigure(0, weight= 1)
        self.root.columnconfigure(1, weight= 1)
        self.root.rowconfigure(0, weight= 1)

        # Creating Left and Right sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons"""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, sticky= 'nw')

        # Labels and Entry Fields
        labels = ["Climber Name", "Category", "Tops (count, attempts)", "Zones (count, attempts)", "Low Zones (count, attempts)"]
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text= label + ":").grid(row=i, column=0, padx=5, pady=5)
            entry = ttk.Entry(self.left_frame)
            entry.grid(row= i, column= 1, padx=5, pady=5, sticky= "w")
            self.entries[label] = entry #stores in dictionary for easy access

            # Climber Name
        ttk.Label(self.left_frame, text="Climber Name:").grid(row=0, column=0, padx=5, pady=5)
        self.name_entry = ttk.Entry(self.left_frame)
        self.name_entry.grid(row=0, column=1, padx=5, pady=5, sticky= "w")

            # Climber Category
        ttk.Label(self.left_frame, text="Category:").grid(row=1, column=0, padx=5, pady=5)
        self.category_entry = ttk.Entry(self.left_frame)
        self.category_entry.grid(row=1, column= 1, padx=5, pady=5, sticky= "w")
        
            # Climber Scores, Tops, Zones, Low Zones
        ttk.Label(self.left_frame, text="Tops (count, attempts):").grid(row=2, column=0, padx=5, pady=5)
        self.tops_entry = ttk.Entry(self.left_frame)
        self.tops_entry.grid(row=2, column= 1, padx=5, pady=5, sticky= "w")
        
        ttk.Label(self.left_frame, text="Zones (count, attempts):").grid(row=3, column=0, padx=5, pady=5)
        self.zones_entry = ttk.Entry(self.left_frame)
        self.zones_entry.grid(row=3, column=1, padx=5, pady=5, sticky= "w")
        
        ttk.Label(self.left_frame, text="Low Zones (count, attempts):").grid(row=4, column=0, padx=5, pady=5)
        self.low_zones_entry = ttk.Entry(self.left_frame)
        self.low_zones_entry.grid(row=4, column=1, padx=5, pady=5, sticky= "w")

        
        # Leftside Buttons
        ttk.Button(self.left_frame, text="Add Climber", command=self.add_climber).grid(row= 5, column= 0, columnspan= 2, pady= 10)
        ttk.Button(self.left_frame, text="Edit Climber").grid(row= 6, column= 0, columnspan= 2, padx= 10, pady= 10, sticky= "w")
        ttk.Button(self.left_frame, text= "Remove Climber").grid(row= 6, column= 1, columnspan= 2, padx= 10, pady= 10, sticky= "e")

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky= 'ne')

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=50)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady=10, sticky= "nsew")

        # Rightside Buttons
        ttk.Button(self.right_frame, text="Clear Leaderboard", command=self.clear_leaderboard_ask).grid(row= 1, column= 0, columnspan= 2, pady= 10)
        ttk.Button(self.right_frame, text="Show Leaderboard", command=self.show_leaderboard).grid(row= 1, column= 1, columnspan= 2, pady= 10)

    # Functions and Logic
    def clear_entries(self):
        """Clears all but category entry fields"""
        self.name_entry.delete(0, tk.END)
        self.tops_entry.delete(0, tk.END)
        self.zones_entry.delete(0, tk.END)
        self.low_zones_entry.delete(0, tk.END)

    def add_climber(self):
        """Adds a climber to the current leaderboard"""
        try:
            name = self.name_entry.get()
            category = self.category_entry.get()
            tops = tuple(map(int, self.entries["Tops (count, attempts)"].get().split(',')))
            zones = tuple(map(int, self.entries["Zones (count, attempts)"].get().split(',')))
            low_zones = tuple(map(int, self.entries["Low Zones (count, attempts)"].get().split(',')) if self.entries["Low Zones (count, attempts)"].get() else (0, 0))

            
            climber = create_climber(name, category, tops, zones, low_zones)
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"{name} added to the leaderboard!")
            self.clear_entries()
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {e}")

    def show_leaderboard(self):
        """Displays the current leaderboard, with rankings"""
        self.leaderboard.rank_climbers()
        self.leaderboard_text.delete(1.0, tk.END)
        for climber in self.leaderboard.climbers:
            self.leaderboard_text.insert(tk.END, f"{climber}\n\n")

    def edit_climber(self):
        """Edits the score of an existing climber"""
        try: 
            name = self.name_entry.get()
            category = self.category_entry.get()
            tops = tuple(map(int, self.tops_entry.get().split(',')))
            zones = tuple(map(int, self.zones_entry.get().split(',')))
            low_zones = tuple(map(int, self.low_zones_entry.get().split(','))) if self.low_zones_entry.get() else (0, 0)

            for climber in self.leaderboard.climbers:
                if climber.name == name:
                    climber.scores.tops = Scores.Top(*tops)
                    climber.scores.zones = Scores.Zone(*zones)
                    climber.scores.low_zones = Scores.LowZone(*low_zones)
                    messagebox.showinfo("Success", f"{name} has been updated.")
                    self.clear_entries()
                    return
        except Exception as e:
            messagebox.showerror("Error", f"invalid input {e}")

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries"""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.clear_leaderboard()
        
    def clear_leaderboard(self):
        """Clears the leaderboard values"""
        self.leaderboard.climbers = []
        self.leaderboard_text.delete(1.0, tk.END)
        messagebox.showinfo("Success", "Leaderboard has been cleared.")

if __name__ == "__main__":
    root = tk.Tk()
    app = ScoringApp(root)
    root.mainloop()


In [45]:
# Leaderboard to the right
class LeaderboardApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        
        # Create Main Frame to Hold Everything
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(expand=True, fill="both", padx=20, pady=20)

        # Create Left and Right Sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons."""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, sticky="nw")

        labels = ["Climber Name:", "Tops (count, attempts):", 
                  "Zones (count, attempts):", "Low Zones (count, attempts):"]
        
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text=label).grid(row=i, column=0, padx=5, pady=5, sticky="e")
            entry = ttk.Entry(self.left_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky="w")
            self.entries[label] = entry  # Store in dictionary for easy access

        # Button to Add Climber (Stays on Left Side)
        ttk.Button(self.left_frame, text="Add Climber").grid(row=len(labels), column=0, columnspan=2, pady=10)

        # Bottom Buttons (Only "Edit" and "Remove" stay on left)
        self.bottom_frame = ttk.Frame(self.left_frame)
        self.bottom_frame.grid(row=len(labels)+1, column=0, columnspan=2, pady=10)

        ttk.Button(self.bottom_frame, text="Edit Climber").grid(row=0, column=0, padx=5)
        ttk.Button(self.bottom_frame, text="Remove Climber").grid(row=0, column=1, padx=5)

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and leaderboard-related buttons."""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky="n")

        # Leaderboard Text Box
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=50)
        self.leaderboard_text.pack(expand=True, fill="both")

        # Buttons Below the Leaderboard
        self.leaderboard_buttons_frame = ttk.Frame(self.right_frame)
        self.leaderboard_buttons_frame.pack(pady=10)

        ttk.Button(self.leaderboard_buttons_frame, text="Show Leaderboard").pack(side="left", padx=5)
        ttk.Button(self.leaderboard_buttons_frame, text="Clear Leaderboard").pack(side="left", padx=5)

if __name__ == "__main__":
    root = tk.Tk()
    app = LeaderboardApp(root)
    root.mainloop()

In [42]:
# Leaderboard below 
class LeaderboardApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        
        # Configure root layout for expansion
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(2, weight=1)  # Make the leaderboard expand

        # Create Frames
        self.create_input_frame()
        self.create_button_frame()
        self.create_leaderboard()
        self.create_bottom_buttons()

    def create_input_frame(self):
        """Create the input fields for climber details."""
        self.input_frame = ttk.Frame(self.root)
        self.input_frame.grid(row=0, column=0, padx=10, pady=10, sticky="ew")

        labels = ["Climber Name:", "Category:", "Tops (count, attempts):", 
                  "Zones (count, attempts):", "Low Zones (count, attempts):"]
        
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.input_frame, text=label).grid(row=i, column=0, padx=5, pady=5, sticky="e")
            entry = ttk.Entry(self.input_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky="w")
            self.entries[label] = entry  # Store in dictionary for easy access

    def create_button_frame(self):
        """Create buttons for adding and showing leaderboard."""
        self.button_frame = ttk.Frame(self.root)
        self.button_frame.grid(row=1, column=0, pady=10)

        ttk.Button(self.button_frame, text="Add Climber").grid(row=0, column=0, padx=5)
        ttk.Button(self.button_frame, text="Show Leaderboard").grid(row=0, column=1, padx=5)

    def create_leaderboard(self):
        """Create the text box for displaying the leaderboard."""
        self.leaderboard_text = tk.Text(self.root, height=15, width=50)
        self.leaderboard_text.grid(row=2, column=0, pady=10, padx=10, sticky="nsew")

    def create_bottom_buttons(self):
        """Create edit and clear buttons at the bottom."""
        self.bottom_frame = ttk.Frame(self.root)
        self.bottom_frame.grid(row=3, column=0, pady=10)

        ttk.Button(self.bottom_frame, text="Edit Climber").grid(row=0, column=0, padx=5)
        ttk.Button(self.bottom_frame, text="Clear Leaderboard").grid(row=0, column=1, padx=5)
        ttk.Button(self.bottom_frame, text="Remove Climber").grid(row=0, column=2, padx=5)

if __name__ == "__main__":
    root = tk.Tk()
    app = LeaderboardApp(root)
    root.mainloop()